In [13]:
import pandas as pd
import seaborn as sns
import os
from googleapiclient.discovery import build

In [14]:
API_KEY = os.environ["YOUTUBE_API_KEY"]
youtube = build("youtube", "v3", developerKey=API_KEY)

In [15]:
def get_channel_id(service, username):
    response = service.channels().list(
        part="contentDetails,statistics",
        forUsername=username
    ).execute()
    return response["items"][0]["id"]

In [16]:
CHANNEL_ID = get_channel_id(youtube, "Felixje0")

In [17]:
def get_upload_id(service, channel_id):
    response = service.channels().list(
        part="contentDetails,statistics",
        id=channel_id
    ).execute()
    return response["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]

In [18]:
UPLOAD_ID = get_upload_id(youtube, CHANNEL_ID)

In [19]:
def get_video_ids(service, upload_id):
    video_ids = []

    response = service.playlistItems().list(
        part="contentDetails",
        playlistId=upload_id,
        maxResults=50
    ).execute()

    for video in response["items"]:
        video_ids.append(video["contentDetails"]["videoId"])
    
    next_page_token = response.get("nextPageToken")
    while next_page_token:
        response = service.playlistItems().list(
            part="contentDetails",
            playlistId=upload_id,
            maxResults=50,
            pageToken=next_page_token
        ).execute()

        for video in response["items"]:
            video_ids.append(video["contentDetails"]["videoId"])

        next_page_token = response.get("nextPageToken")

    return video_ids

In [20]:
video_ids = get_video_ids(youtube, UPLOAD_ID)
len(video_ids)

3819